In [1]:
import numpy as np
import random

In [2]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)


In [3]:
def clip(gradients, max_value):
    for key in gradients.keys():
        gradients[key] = np.clip(gradients[key], -max_value, max_value)
    return gradients


In [4]:
def sample(parameters, char_to_ix, seed, start_char=None):
    
    # Retrieve parameters and relevant shapes from "parameters" dictionary
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]

    # Initialize hidden state and input
    a_prev = np.zeros((n_a, 1))
    x = np.zeros((vocab_size, 1))

    # If a starting character is provided, initialize the one-hot vector
    if start_char:
        x[char_to_ix[start_char]] = 1

    # Initialize output variables
    indices = []
    idx = -1  # Initialize index to a value that is not '\n'
    counter = 0
    newline_character = char_to_ix['\n']

    # Sampling loop
    while (idx != newline_character and counter != 50):
        # Forward propagate through the RNN
        a = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + b)
        z = np.dot(Wya, a) + by
        y = softmax(z)

        # Sample a character index from the probability distribution
        np.random.seed(counter + seed)
        idx = np.random.choice(range(vocab_size), p=y.ravel())

        # Append the index to the list
        indices.append(idx)

        # Update the input x with the sampled index
        x = np.zeros((vocab_size, 1))
        x[idx] = 1

        # Update the hidden state
        a_prev = a

        counter += 1

    # Append a newline character if the maximum length is reached
    if counter == 50:
        indices.append(newline_character)

    return indices


In [5]:
def optimize(X, Y, a_prev, parameters, learning_rate):
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    gradients, a_last = rnn_backward(X, Y, parameters, cache)
    gradients = clip(gradients, 5)
    parameters = update_parameters(parameters, gradients, learning_rate)
    return loss, gradients, a_last


In [6]:
def rnn_forward(X, Y, a_prev, parameters):
    Waa, Wax, Wya, b, by = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['b'], parameters['by']
    vocab_size, n_a = by.shape[0], Waa.shape[1]
    x, a, y_hat = {}, {}, {}
    a[-1] = a_prev
    loss = 0

    for t in range(len(X)):
        x[t] = np.zeros((vocab_size, 1))
        if X[t] is not None:
            x[t][X[t]] = 1
        a[t] = np.tanh(np.dot(Wax, x[t]) + np.dot(Waa, a[t-1]) + b)
        y_hat[t] = softmax(np.dot(Wya, a[t]) + by)
        loss -= np.log(y_hat[t][Y[t], 0])

    cache = (x, a, y_hat)
    return loss, cache


In [7]:


def rnn_backward(X, Y, parameters, cache):
    Waa, Wax, Wya, b, by = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['b'], parameters['by']
    x, a, y_hat = cache
    gradients = {}
    dx, da_prev = {}, np.zeros_like(a[0])
    gradients['dWax'], gradients['dWaa'], gradients['dWya'] = np.zeros_like(Wax), np.zeros_like(Waa), np.zeros_like(Wya)
    gradients['db'], gradients['dby'] = np.zeros_like(b), np.zeros_like(by)

    for t in reversed(range(len(X))):
        dy = np.copy(y_hat[t])
        dy[Y[t]] -= 1
        gradients['dWya'] += np.dot(dy, a[t].T)
        gradients['dby'] += dy
        da = np.dot(Wya.T, dy) + da_prev
        dz = (1 - a[t] ** 2) * da
        gradients['db'] += dz
        gradients['dWax'] += np.dot(dz, x[t].T)
        gradients['dWaa'] += np.dot(dz, a[t-1].T)
        da_prev = np.dot(Waa.T, dz)

    return gradients, a[len(X)-1]


In [8]:
def update_parameters(parameters, gradients, learning_rate):
    for key in parameters.keys():
        parameters[key] -= learning_rate * gradients['d' + key]
    return parameters


In [9]:
def initialize_parameters(n_a, n_x, n_y):
    np.random.seed(1)
    parameters = {
        'Wax': np.random.randn(n_a, n_x) * 0.01,
        'Waa': np.random.randn(n_a, n_a) * 0.01,
        'Wya': np.random.randn(n_y, n_a) * 0.01,
        'b': np.zeros((n_a, 1)),
        'by': np.zeros((n_y, 1))
    }
    return parameters


In [10]:




def model(data, ix_to_char, char_to_ix, num_iterations=35000, n_a=50, dino_names=7, vocab_size=27):
    parameters = initialize_parameters(n_a, vocab_size, vocab_size)
    loss = -np.log(1.0 / vocab_size) * dino_names
    with open(data) as f:
        examples = f.readlines()
    examples = [x.lower().strip() for x in examples]
    np.random.seed(0)
    np.random.shuffle(examples)
    a_prev = np.zeros((n_a, 1))

    for j in range(num_iterations):
        index = j % len(examples)
        X = [None] + [char_to_ix[ch] for ch in examples[index]]
        Y = X[1:] + [char_to_ix['\n']]
        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters, 0.01)
        loss = loss * 0.999 + curr_loss * 0.001

        if j % 2000 == 0:
            print(f"Iteration {j}, loss: {loss}")
            for name in range(dino_names):
                sampled_indices = sample(parameters, char_to_ix, name)
                print(''.join(ix_to_char[ix] for ix in sampled_indices), end='\n')
            print('\n')

    return parameters


In [11]:

#data = "dinos.txt"
#with open(data, 'r') as f:
#    text = f.read().lower()
#chars = sorted(list(set(text)))
#char_to_ix = {ch: i for i, ch in enumerate(chars)}
#ix_to_char = {i: ch for i, ch in enumerate(chars)}
#
#parameters = model(data, ix_to_char, char_to_ix)


In [12]:
def user_generate(parameters, char_to_ix, ix_to_char):
   
    while True:
        user_input = input("Enter the starting character (or type 'exit' to quit): ").strip().lower()
        
        if user_input == 'exit':  
            print("Exiting the generator. Goodbye!")
            break
        
        if len(user_input) != 1 or user_input not in char_to_ix:
            print("Please enter a single valid character from the dataset.")
            continue
        
        # Initialize input vector with the user's starting character
        seed_char = user_input
        seed_idx = char_to_ix[seed_char]
        x = np.zeros((len(char_to_ix), 1))  # One-hot encoding of the starting character
        x[seed_idx] = 1
        
        # Sampling based on the user input
        print(f"Generating a name starting with '{seed_char}':")
        indices = sample(parameters, char_to_ix, seed=0)  # Call sample function
        
        # Convert indices to characters and print the result
        generated_name = ''.join([ix_to_char[idx] for idx in indices])
        print(f"Generated Name: {generated_name}\n")




In [13]:

data = "shakes_words.txt"
with open(data, 'r') as f:
    text = f.read().lower()
chars = sorted(list(set(text)))
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

parameters = model(data, ix_to_char, char_to_ix)


Iteration 0, loss: 23.070856391082994
nkknzexbw

kknzexbw

knzexbw

nzexbw

zexbw

exbw

xbw



Iteration 2000, loss: 22.554798418388856
lemivet

henwbtat

inwbtat

lvet

wet

ds

tat



Iteration 4000, loss: 20.889635148453504
ngerwat

inives

jmuhs

nwbtat

wct

ds

ucs



Iteration 6000, loss: 19.987230721979735
nghives

jlives

lives

nvet

wet

ds

tat



Iteration 8000, loss: 19.491186661671833
ongives

khives

lives

owes

watat

drat

ubs



Iteration 10000, loss: 19.20993927274358
nferwes

inives

kivet

nvat

wauct

cuat

ubu



Iteration 12000, loss: 19.03590401724015
menguit

giovat

hives

mugs

wauct

ctat

ubt



Iteration 14000, loss: 18.601997418226578
menktes

glives

hives

mugt

wetat

cuct

uct



Iteration 16000, loss: 18.785961552305363
oninuit

inkulucu

jowet

owetat

watat

cuds

uct



Iteration 18000, loss: 18.822064083190423
neleudr

heowat

inves

nues

watas

crat

uat



Iteration 20000, loss: 18.5393609176628
menives

fiowes

gives

muhs

wetat

ctat

u

In [14]:
#user_generate(parameters, char_to_ix, ix_to_char)

In [15]:
def sample(parameters, char_to_ix, seed, start_char=None):
    
    # Retrieve parameters and relevant shapes from "parameters" dictionary
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]

    # Initialize hidden state and input
    a_prev = np.zeros((n_a, 1))
    x = np.zeros((vocab_size, 1))

    # If a starting character is provided, initialize the one-hot vector
    if start_char:
        x[char_to_ix[start_char]] = 1  # Set the input to the one-hot encoded vector for start_char

    # Initialize output variables
    indices = []
    idx = -1  # Initialize index to a value that is not '\n'
    counter = 0
    newline_character = char_to_ix['\n']

    # Sampling loop
    while (idx != newline_character and counter != 50):
        # Forward propagate through the RNN
        a = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + b)
        z = np.dot(Wya, a) + by
        y = softmax(z)

        # Sample a character index from the probability distribution
        np.random.seed(counter + seed)  # Ensure random seed is influenced by counter
        idx = np.random.choice(range(vocab_size), p=y.ravel())

        # Append the index to the list
        indices.append(idx)

        # Update the input x with the sampled index
        x = np.zeros((vocab_size, 1))
        x[idx] = 1

        # Update the hidden state
        a_prev = a

        counter += 1

    # Append a newline character if the maximum length is reached
    if counter == 50:
        indices.append(newline_character)

    return indices

def user_generate(parameters, char_to_ix, ix_to_char):
    while True:
        user_input = input("Enter the starting character (or type 'exit' to quit): ").strip().lower()

        if user_input == 'exit':  
            print("Exiting the generator. Goodbye!")
            break

        if len(user_input) != 1 or user_input not in char_to_ix:
            print("Please enter a single valid character from the dataset.")
            continue

        # Initialize input vector with the user's starting character
        seed_char = user_input
        seed_idx = char_to_ix[seed_char]
        x = np.zeros((len(char_to_ix), 1))  # One-hot encoding of the starting character
        x[seed_idx] = 1
        
        # Sampling based on the user input
        print(f"Generating a name starting with '{seed_char}':")
        indices = sample(parameters, char_to_ix, seed=0, start_char=seed_char)  # Pass start_char to the sample function
        
        # Convert indices to characters and print the result
        generated_name = ''.join([ix_to_char[idx] for idx in indices])
        print(f"Generated Name: {generated_name}\n")

# Assuming your data and model setup is correct
data = "shakes_words.txt"
with open(data, 'r') as f:
    text = f.read().lower()

chars = sorted(list(set(text)))
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

    

parameters = model(data, ix_to_char, char_to_ix)

Iteration 0, loss: 23.070856391082994
nkknzexbw

kknzexbw

knzexbw

nzexbw

zexbw

exbw

xbw



Iteration 2000, loss: 22.554798418388856
lemivet

henwbtat

inwbtat

lvet

wet

ds

tat



Iteration 4000, loss: 20.889635148453504
ngerwat

inives

jmuhs

nwbtat

wct

ds

ucs



Iteration 6000, loss: 19.987230721979735
nghives

jlives

lives

nvet

wet

ds

tat



Iteration 8000, loss: 19.491186661671833
ongives

khives

lives

owes

watat

drat

ubs



Iteration 10000, loss: 19.20993927274358
nferwes

inives

kivet

nvat

wauct

cuat

ubu



Iteration 12000, loss: 19.03590401724015
menguit

giovat

hives

mugs

wauct

ctat

ubt



Iteration 14000, loss: 18.601997418226578
menktes

glives

hives

mugt

wetat

cuct

uct



Iteration 16000, loss: 18.785961552305363
oninuit

inkulucu

jowet

owetat

watat

cuds

uct



Iteration 18000, loss: 18.822064083190423
neleudr

heowat

inves

nues

watas

crat

uat



Iteration 20000, loss: 18.5393609176628
menives

fiowes

gives

muhs

wetat

ctat

u

In [17]:
user_generate(parameters, char_to_ix, ix_to_char)

Enter the starting character (or type 'exit' to quit):  n


Generating a name starting with 'n':
Generated Name: egint




Enter the starting character (or type 'exit' to quit):  t


Generating a name starting with 't':
Generated Name: ing




Enter the starting character (or type 'exit' to quit):  o


...

Enter the starting character (or type 'exit' to quit):  l


Generating a name starting with 'l':
Generated Name: igheter




Enter the starting character (or type 'exit' to quit):  i


Generating a name starting with 'i':
Generated Name: ng




KeyboardInterrupt: Interrupted by user